In [1]:
#This is a data-dig test bench on laptop with jupyter, check out at 20230408 2257

import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

from sqlalchemy import create_engine

source_df = pd.read_csv(".\datadig\exp\exp1.csv")
print(source_df)


          日期        业务类型   笔数         交易金额       收入       成本
0   20230310    网关与支付单报送  588    514361.36  3282.95  1034.59
1   20230310     跨境人民币付款    7   2584404.70  2165.22   250.00
2   20230310     跨境人民币收款    4  18960768.32  4413.77     0.00
3   20230311    网关与支付单报送  774    163472.02  1018.21   332.24
4   20230312    网关与支付单报送  419    102350.41   672.91   210.13
..       ...         ...  ...          ...      ...      ...
59  20230330     跨境人民币收款    5  26848886.40  9064.04     0.00
60  20230330  跨境付款(离岸换汇)    1    481188.00  5705.65   215.25
61  20230331    网关与支付单报送  672    212862.56  1390.75   428.17
62  20230331     跨境人民币付款   13   4858342.90  3637.05   778.62
63  20230331     跨境人民币收款    4  22273473.12  6980.73     0.00

[64 rows x 6 columns]


In [12]:
#connect to pgdb and use sql to get data

import psycopg2
# Connect to PostgreSQL database
pg_conn = psycopg2.connect(
    host="10.66.5.20",
    database="analysis_db",
    user="data_analysis",
    password="data_analysis"
)

# SQL query to extract data from PostgreSQL
pg_query = "select total_trans_amt, total_income_amt, total_cost_amt, total_refund_amt, data_source from anl_newpay_management_summary where data_source = '010003' and source_trans_date = '20230404';"

# Use pandas to read PostgreSQL data into a dataframe
pg_df_so = pd.read_sql(pg_query, pg_conn)

pg_df_so.head()


,total_trans_amt,total_income_amt,total_cost_amt,total_refund_amt,data_source
0,80971687.25,29144.01,1693.39,23129.56,010003


It can also use sqlalchemy to connect database, and use sql to get data into frame
It's a hint below:


In [ ]:
#推荐使用SQLAlchemy，需要先安装SQLAlchemy库
#新版本的pandas库中con参数使用sqlalchemy库创建的create_engine对象 。创建create_engine对象(格式类似于URL地址)
#
'''
# import necessary packages
import pandas
import psycopg2
from sqlalchemy import create_engine
  
# establish connection with the database
engine = create_engine(
    "dialect+driver//username:password@hostname:portnumber/databasename")
  
# read the pandas dataframe
data = pandas.read_csv("path to dataset")
  
# connect the pandas dataframe with postgresql table
data.to_sql('loan_data', engine, if_exists='replace')
'''
#from sqlalchemy import create_engine
#import psycopg2
#import pandas as pd
#from IPython.display import display

#pg db config
PG_HOST = '10.66.5.20'
PG_PORT = '5432'
PG_USER = 'data_analysis'
PG_PASSWD = 'data_analysis'
PG_DB = 'analysis_db'

url = f"postgresql://{PG_USER}:{PG_PASSWD}@{PG_HOST}:{PG_PORT}/{PG_DB}"

#engine = create_engine(url, pool_size = 50, echo=False)
engine = create_engine(url)


In [ ]:
# SQL query to extract data from PostgreSQL
pg_query = "select source_trans_date, total_trans_amt, total_income_amt, total_cost_amt, total_refund_amt, data_source from anl_newpay_management_summary where data_source = '010003' and source_trans_date in ('20230401','20230402','20230403','20230404');"

# Use pandas to read PostgreSQL data into a dataframe
pg_df = pd.read_sql(pg_query, engine)

# Print the dataframe
#print(pg_df)
#pg_df.tail()
display(pg_df)


It can also create a def of course...

In [12]:
#dataframe version 1
#source dataframe

source_df = source_df.rename(columns={u'日期': '日期', u'业务类型': '业务类型', u'笔数': '交易笔数', u'交易金额': '交易金额', u'收入': '收入', u'成本': '成本'})
source_df['毛利'] = source_df['收入'] - source_df['成本'] # Add a new column for gross profit
#display(source_df)

table_cross = pd.pivot_table(source_df, values=[u'交易金额', u'交易笔数', u'收入', u'成本', '毛利'], index=[ u'日期', u'业务类型'])
display(table_cross)

# 查找全部所有业务类型，以每天不全缺失的业务类型，最后其他的列补0
all_types = table_cross.index.get_level_values('业务类型').unique()
all_dates = table_cross.index.get_level_values('日期').unique()
table_cross = table_cross.reindex(index=pd.MultiIndex.from_product([all_dates, all_types]), fill_value=0)
display(table_cross)

'''
# 用pivot_table方法将日期和业务类型作为index调整数据
order1_df = pd.pivot_table(source_df, index=['日期', '业务类型'])

# 将日期从字符转换为时间格式
order1_df['日期'] = pd.to_datetime(order1_df['日期'])

# 查找全部所有业务类型，以每天不全缺失的业务类型，最后其他的列补0
all_types = order1_df.index.get_level_values('业务类型').unique()
all_dates = order1_df.index.get_level_values('日期').unique()
order1_df = order1_df.reindex(index=pd.MultiIndex.from_product([all_dates, all_types]), fill_value=0)

# 分别再按照日期与业务类型进行排序
order1_df = order1_df.sort_values(by=['日期', '业务类型'])



order1_df = (
    source_df.pivot_table(index=['日期', '业务类型'])
             .assign(日期=lambda x: pd.to_datetime(x['日期']))
             .reset_index()
             .pivot(index='日期', columns='业务类型', values='销售额')
             .fillna(0)
             .sort_index()
             .sort_index(axis=1)
)


display(order1_df)
'''

交易笔数         交易金额       成本       收入       毛利
日期       业务类型                                                    
20230310 网关与支付单报送     588    514361.36  1034.59  3282.95  2248.36
         跨境人民币付款        7   2584404.70   250.00  2165.22  1915.22
         跨境人民币收款        4  18960768.32     0.00  4413.77  4413.77
20230311 网关与支付单报送     774    163472.02   332.24  1018.21   685.97
20230312 网关与支付单报送     419    102350.41   210.13   672.91   462.78
...                   ...          ...      ...      ...      ...
20230330 跨境人民币收款        5  26848886.40     0.00  9064.04  9064.04
         跨境付款(离岸换汇)     1    481188.00   215.25  5705.65  5490.40
20230331 网关与支付单报送     672    212862.56   428.17  1390.75   962.58
         跨境人民币付款       13   4858342.90   778.62  3637.05  2858.43
         跨境人民币收款        4  22273473.12     0.00  6980.73  6980.73

[64 rows x 5 columns]

交易笔数         交易金额       成本       收入       毛利
日期       业务类型                                                    
20230310 网关与支付单报送     588    514361.36  1034.59  3282.95  2248.36
         跨境人民币付款        7   2584404.70   250.00  2165.22  1915.22
         跨境人民币收款        4  18960768.32     0.00  4413.77  4413.77
         跨境付款(离岸换汇)     0         0.00     0.00     0.00     0.00
20230311 网关与支付单报送     774    163472.02   332.24  1018.21   685.97
...                   ...          ...      ...      ...      ...
20230330 跨境付款(离岸换汇)     1    481188.00   215.25  5705.65  5490.40
20230331 网关与支付单报送     672    212862.56   428.17  1390.75   962.58
         跨境人民币付款       13   4858342.90   778.62  3637.05  2858.43
         跨境人民币收款        4  22273473.12     0.00  6980.73  6980.73
         跨境付款(离岸换汇)     0         0.00     0.00     0.00     0.00

[88 rows x 5 columns]

"\n# 用pivot_table方法将日期和业务类型作为index调整数据\norder1_df = pd.pivot_table(source_df, index=['日期', '业务类型'])\n\n# 将日期从字符转换为时间格式\norder1_df['日期'] = pd.to_datetime(order1_df['日期'])\n\n# 查找全部所有业务类型，以每天不全缺失的业务类型，最后其他的列补0\nall_types = order1_df.index.get_level_values('业务类型').unique()\nall_dates = order1_df.index.get_level_values('日期').unique()\norder1_df = order1_df.reindex(index=pd.MultiIndex.from_product([all_dates, all_types]), fill_value=0)\n\n# 分别再按照日期与业务类型进行排序\norder1_df = order1_df.sort_values(by=['日期', '业务类型'])\n\n\n\norder1_df = (\n    source_df.pivot_table(index=['日期', '业务类型'])\n             .assign(日期=lambda x: pd.to_datetime(x['日期']))\n             .reset_index()\n             .pivot(index='日期', columns='业务类型', values='销售额')\n             .fillna(0)\n             .sort_index()\n             .sort_index(axis=1)\n)\n\n\ndisplay(order1_df)\n"

In [21]:
#dataframe version 2

#table_cross_v2 = pd.pivot_table(source_df, values=[u'交易金额', u'交易笔数', u'收入', u'成本', '毛利'], index=[ u'日期', u'业务类型'])
#table_cross_2 = pd.pivot_table(table_cross.reset_index(), values=[u'交易金额', u'交易笔数', u'收入', u'成本', '毛利'], index=[ u'日期'], columns=['业务类型'], aggfunc=sum)
table_cross_2_1 = pd.pivot_table(table_cross.reset_index(), values=[u'交易金额'], index=[ u'日期'], columns=['业务类型'])
display(table_cross_2_1)


交易金额                                    
业务类型       网关与支付单报送     跨境人民币付款      跨境人民币收款 跨境付款(离岸换汇)
日期                                                     
20230310  514361.36  2584404.70  18960768.32       0.00
20230311  163472.02        0.00         0.00       0.00
20230312  102350.41        0.00         0.00       0.00
20230313  176900.27  6527124.01  29327240.00  232151.38
20230314  185706.32  2579032.45  14156215.70       0.00
20230315  261790.16   978133.93  36117938.01       0.00
20230316  199109.04  3451229.44  30473300.00  601081.61
20230317  164434.71  2799947.85  15352940.00       0.00
20230318  180129.00        0.00         0.00       0.00
20230319   93788.19        0.00         0.00       0.00
20230320  146208.01  3097576.95  12440047.15   80255.60
20230321   99805.94  2116624.29  29184468.14  910328.50
20230322  202172.10   984847.46  38026395.81  708569.51
20230323  129992.51  6600297.06  13907000.00   59880.95
20230324   86429.43  1224191.20  23669213.43       0.00
20230325  228196.46        0.00         0.00       0.00
20230326  208010.29        0.00         0.00       0.00
20230327  159124.58  6770820.87  21029855.08  199780.40
20230328  211365.48  1554861.46  39933826.44   71953.00
20230329  120629.81   850351.46  52663798.04  495049.50
20230330  197882.38  2297008.28  26848886.40  481188.00
20230331  212862.56  4858342.90  22273473.12       0.00

In [35]:
table_cross_2_2 = pd.pivot_table(table_cross.reset_index(), values=[u'交易笔数', u'交易金额', u'收入', u'成本', '毛利'], index=[ u'日期'], aggfunc=sum)
display(table_cross_2_2)

,交易笔数,交易金额,成本,收入,毛利
日期,,,,,
20230310,599,22059534.38,1284.59,9861.94,8577.35
20230311,774,163472.02,332.24,1018.21,685.97
20230312,419,102350.41,210.13,672.91,462.78
20230313,680,36263415.66,1211.48,15839.48,14628.00
20230314,819,16920954.47,973.74,7321.65,6347.91
20230315,1201,37357862.10,822.73,9558.72,8735.99
20230316,774,34724720.09,1314.19,16943.77,15629.58
20230317,595,18317322.56,893.05,8391.53,7498.48
20230318,823,180129.00,364.37,1016.36,651.99


In [59]:
table_cross_2_3 = pd.concat([table_cross_2_1, table_cross_2_2], axis=1)
display(table_cross_2_3)


,"(交易金额, 网关与支付单报送)","(交易金额, 跨境人民币付款)","(交易金额, 跨境人民币收款)","(交易金额, 跨境付款(离岸换汇))",交易笔数,交易金额,成本,收入,毛利
日期,,,,,,,,,
20230310,514361.36,2584404.70,18960768.32,0.00,599,22059534.38,1284.59,9861.94,8577.35
20230311,163472.02,0.00,0.00,0.00,774,163472.02,332.24,1018.21,685.97
20230312,102350.41,0.00,0.00,0.00,419,102350.41,210.13,672.91,462.78
20230313,176900.27,6527124.01,29327240.00,232151.38,680,36263415.66,1211.48,15839.48,14628.00
20230314,185706.32,2579032.45,14156215.70,0.00,819,16920954.47,973.74,7321.65,6347.91
20230315,261790.16,978133.93,36117938.01,0.00,1201,37357862.10,822.73,9558.72,8735.99
20230316,199109.04,3451229.44,30473300.00,601081.61,774,34724720.09,1314.19,16943.77,15629.58
20230317,164434.71,2799947.85,15352940.00,0.00,595,18317322.56,893.05,8391.53,7498.48
20230318,180129.00,0.00,0.00,0.00,823,180129.00,364.37,1016.36,651.99


In [60]:
#table_cross_2_3 = table_cross_2_3.reindex(columns=[u'笔数', u'交易量', '网关与支付单报送', '跨境人民币付款', '跨境人民币收款', '跨境付款(离岸换汇)', u'收入', u'成本', '毛利'])

columns = list(table_cross_2_3)
print(columns)

table_cross_2_3 = table_cross_2_3.rename(columns={u'(交易金额, 网关与支付单报送)': '网关与支付单报送', u'(交易金额, 跨境人民币付款)': '跨境人民币付款', u'(交易金额, 跨境人民币收款)': '跨境人民币收款', u'(交易金额, 跨境付款(离岸换汇))': '跨境付款(离岸换汇)', u'交易笔数': '笔数', u'交易金额': '交易量'})
columns = list(table_cross_2_3)
print(columns)

#It seems doesn't change, could be some charset problem...

[('交易金额', '网关与支付单报送'), ('交易金额', '跨境人民币付款'), ('交易金额', '跨境人民币收款'), ('交易金额', '跨境付款(离岸换汇)'), '交易笔数', '交易金额', '成本', '收入', '毛利']
[('交易金额', '网关与支付单报送'), ('交易金额', '跨境人民币付款'), ('交易金额', '跨境人民币收款'), ('交易金额', '跨境付款(离岸换汇)'), '笔数', '交易量', '成本', '收入', '毛利']


In [61]:
#table_cross_2_3 = table_cross_2_3.rename(columns={'(交易金额, 网关与支付单报送)': '网关与支付单报送'})
#table_cross_2_4 = table_cross_2_3.rename(columns={'\(交易金额, 网关与支付单报送\)': '网关与支付单报送'})
#table_cross_2_4.columns = table_cross_2_4.columns.str.replace(r"\(|\)", "")

#cols = list(table_cross_2_4)
#print(cols)

col = table_cross_2_3.columns
display(col)

#table_cross_2_3.columns = list("'网关与支付单报送''跨境人民币付款''跨境人民币收款''跨境付款(离岸换汇)''笔数''交易量''成本''收入''毛利'")
#col = table_cross_2_3.columns
#display(col)
#display(table_cross_2_3)




Index([  ('交易金额', '网关与支付单报送'),    ('交易金额', '跨境人民币付款'),    ('交易金额', '跨境人民币收款'),
       ('交易金额', '跨境付款(离岸换汇)'),                   '笔数',                  '交易量',
                         '成本',                   '收入',                   '毛利'],
      dtype='object')

In [11]:

""" 
import pandas as pd
from sqlalchemy import create_engine

#pg db config
PG_HOST = '10.66.5.20'
PG_PORT = '5432'
PG_USER = 'data_analysis'
PG_PASSWD = 'data_analysis'
PG_DB = 'analysis_db'

url = "postgresql://{PG_USER}:{PG_PASSWD}@{PG_HOST}:{PG_PORT}/{PG_DB}"

#engine = create_engine(url, pool_size = 50, echo=False)
engine = create_engine(url) 
"""

import pandas as pd
from sqlalchemy import create_engine

def query_database(sql):
    # PostgreSQL database configuration
    PG_HOST = '10.66.5.20'
    PG_PORT = '5432'
    PG_USER = 'data_analysis'
    PG_PASSWD = 'data_analysis'
    PG_DB = 'analysis_db'

    # Create engine object to connect to PostgreSQL database
    url = f"postgresql://{PG_USER}:{PG_PASSWD}@{PG_HOST}:{PG_PORT}/{PG_DB}"
    engine = create_engine(url)

    # Use pandas to read PostgreSQL data into a dataframe
    df = pd.read_sql(sql, engine)

    return df


# SQL query to extract data from PostgreSQL
pg_query_2 = """
SELECT 
    source_trans_date as 日期,
    COALESCE(sum(trans_count), 0) as 笔数,
    COALESCE(SUM(total_trans_amt), 0) as 交易额,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_1' THEN total_trans_amt ELSE 0 END), 0) as 跨境付款（离岸换汇）,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_2' THEN total_trans_amt ELSE 0 END), 0) as 跨境人民币付款,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_3' THEN total_trans_amt ELSE 0 END), 0) as 跨境人民币收款,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_4' THEN total_trans_amt ELSE 0 END), 0) as 网关与支付单报送,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_5' THEN total_trans_amt ELSE 0 END), 0) as 网关b2b支付,
    sum(total_commission_amt + total_other_income_amt + total_income_amt) as 收入,
    sum(total_cost_amt) as 成本,
    sum(total_commission_amt + total_other_income_amt + total_income_amt - total_cost_amt) as 毛利
FROM anl_cross_business_group_by 
WHERE 
    source_trans_date LIKE '202303%'
GROUP BY 
    source_trans_date
ORDER BY
    source_trans_date 
    ;
"""

# Use pandas to read PostgreSQL data into a dataframe
# pg_df_2 = pd.read_sql(pg_query_2, engine)

# Print the dataframe
# print(pg_df)
# pg_df.tail()
# display(pg_df_2)

pg_df_2 = query_database(pg_query_2)

display(pg_df_2)

TypeError: 'dict' object does not support indexing

I don't know why this didn't work with sqlalchemy...
looks like it's better to input a simple sql...
There is a error return:
    TypeError: 'dict' object does not support indexing

In [13]:
#connect to pgdb and use sql to get data

import psycopg2
# Connect to PostgreSQL database
pg_conn = psycopg2.connect(
    host="10.66.5.20",
    database="analysis_db",
    user="data_analysis",
    password="data_analysis"
)

# SQL query to extract data from PostgreSQL
pg_query_2 = """
SELECT 
    source_trans_date as 日期,
    COALESCE(sum(trans_count), 0) as 笔数,
    COALESCE(SUM(total_trans_amt), 0) as 交易额,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_1' THEN total_trans_amt ELSE 0 END), 0) as 跨境付款（离岸换汇）,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_2' THEN total_trans_amt ELSE 0 END), 0) as 跨境人民币付款,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_3' THEN total_trans_amt ELSE 0 END), 0) as 跨境人民币收款,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_4' THEN total_trans_amt ELSE 0 END), 0) as 网关与支付单报送,
    COALESCE(SUM(CASE WHEN business_type = 'CROSS_1_5' THEN total_trans_amt ELSE 0 END), 0) as 网关b2b支付,
    sum(total_commission_amt + total_other_income_amt + total_income_amt) as 收入,
    sum(total_cost_amt) as 成本,
    sum(total_commission_amt + total_other_income_amt + total_income_amt - total_cost_amt) as 毛利
FROM anl_cross_business_group_by 
WHERE 
    source_trans_date LIKE '202303%'
GROUP BY 
    source_trans_date
ORDER BY
    source_trans_date 
    ;
"""

# Use pandas to read PostgreSQL data into a dataframe
pg_df_2 = pd.read_sql(pg_query_2, pg_conn)

display(pg_df_2)


,日期,笔数,交易额,跨境付款（离岸换汇）,跨境人民币付款,跨境人民币收款,网关与支付单报送,网关b2b支付,收入,成本,毛利
0,20230310,599.0,22059534.38,0.00,2584404.70,18960768.32,514361.36,0.0,9861.94,1284.59,8577.35
1,20230311,774.0,163472.02,0.00,0.00,0.00,163472.02,0.0,1018.21,332.24,685.97
2,20230312,419.0,102350.41,0.00,0.00,0.00,102350.41,0.0,672.91,210.13,462.78
3,20230313,680.0,36263415.66,232151.38,6527124.01,29327240.00,176900.27,0.0,15839.48,1211.48,14628.00
4,20230314,819.0,16920954.47,0.00,2579032.45,14156215.70,185706.32,0.0,7321.65,973.74,6347.91
5,20230315,1201.0,37357862.10,0.00,978133.93,36117938.01,261790.16,0.0,9558.72,822.73,8735.99
6,20230316,774.0,34724720.09,601081.61,3451229.44,30473300.00,199109.04,0.0,16943.77,1314.19,15629.58
7,20230317,595.0,18317322.56,0.00,2799947.85,15352940.00,164434.71,0.0,8391.53,893.05,7498.48
8,20230318,823.0,180129.00,0.00,0.00,0.00,180129.00,0.0,1016.36,364.37,651.99
9,20230319,350.0,93788.19,0.00,0.00,0.00,93788.19,0.0,597.94,190.02,407.92


But with pycopg2, we can finish the table...
I don't know why?
maybe I need more test in different ways...

In [25]:
#将没有的日期进行补全

# 查找全部所有业务类型，以每天不全缺失的业务类型，最后其他的列补0
# all_types = table_cross.index.get_level_values('业务类型').unique()
# all_dates = table_cross.index.get_level_values('日期').unique()

#本月days的series,version 1
""" import pandas as pd
import datetime

current_month = datetime.datetime.now().replace(day=1)


# Assuming `current_month` is a datetime object representing the current month
start_date = current_month.replace(day=1)
end_date = current_month.replace(day=pd.Period(current_month, 'M').days_in_month)
dates = pd.date_range(start_date, end_date)
all_dates = [date.strftime('%Y%m%d') for date in dates]
 """

#本月days的series,version 2
""" import pandas as pd
import datetime

current_month = datetime.datetime.now().replace(day=1)

# Assuming `current_month` is a datetime object representing the current month
start_date = current_month.replace(day=1)
end_date = current_month.replace(day=pd.Period(current_month, 'M').days_in_month)
dates = pd.date_range(start_date, end_date)
all_dates = [date.strftime('%Y%m%d') for date in dates]

# Add the previous day of each month to the list
for date in dates:
    prev_day = date - pd.Timedelta(days=1)
    all_dates.append(prev_day.strftime('%Y%m%d')) """

#本月days的series,version 3
import pandas as pd
import datetime

current_month = datetime.datetime.now().replace(day=1)

# Assuming `current_month` is a datetime object representing the current month
start_date = current_month.replace(day=1)
end_date = current_month.replace(day=pd.Period(current_month, 'M').days_in_month)
dates = pd.date_range(start_date, end_date)
all_dates = [date.strftime('%Y%m%d') for date in dates]

# Add the previous day of each month to the list
for date in dates:
    prev_day = date - pd.Timedelta(days=1)
    all_dates.append(prev_day.strftime('%Y%m%d'))

# Add the last day of the previous month to the list
prev_month = current_month - pd.DateOffset(months=1)
last_day_prev_month = prev_month.replace(day=pd.Period(prev_month, 'M').days_in_month)
all_dates.append(last_day_prev_month.strftime('%Y%m%d'))

all_dates = list(set(all_dates))
all_dates.sort()
print(all_dates)


# all_dates.sort(reverse=True)
# print(all_dates)


# pg_df_2.set_index('date', inplace=True)
# pg_df_2 = pg_df_2.reindex(index=dates, fill_value=0)
# pg_df_2 = pg_df_2.reset_index()

'''
pg_df_2 = pg_df_2.reindex(index=pd.MultiIndex.from_product([all_dates, all_types]), fill_value=0)
display(pg_df_2)
'''

['20230331', '20230401', '20230402', '20230403', '20230404', '20230405', '20230406', '20230407', '20230408', '20230409', '20230410', '20230411', '20230412', '20230413', '20230414', '20230415', '20230416', '20230417', '20230418', '20230419', '20230420', '20230421', '20230422', '20230423', '20230424', '20230425', '20230426', '20230427', '20230428', '20230429', '20230430']


'\npg_df_2 = pg_df_2.reindex(index=pd.MultiIndex.from_product([all_dates, all_types]), fill_value=0)\ndisplay(pg_df_2)\n'